In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
driver = webdriver.Chrome()

In [4]:
url = "https://www.flipkart.com/toys/~best-selling-toys/pr?sid=tng"
driver.get(url)

In [5]:
try:
    close_button = driver.find_element(By.XPATH, "//button[contains(text(), '✕')]")
    close_button.click()
except:
    pass

In [6]:
time.sleep(2)

In [7]:
product_names = []
product_prices = []
product_ratings = []
num_reviews = []
discounts = []

In [8]:
def scrape_page():
    products = driver.find_elements(By.CLASS_NAME, "slAVV4")
    for product in products:
        try:
            # Product Name
            name = product.find_element(By.CLASS_NAME, "wjcEIp").get_attribute("title")
            product_names.append(name)
        except:
            product_names.append(None)
        try:
            # Product Price
            price = product.find_element(By.CLASS_NAME, "Nx9bqj").text  # Current price
            product_prices.append(price)
        except:
            product_prices.append(None)
 
        try:
            # Product Rating
            rating = product.find_element(By.CLASS_NAME, "XQDdHH").text
            product_ratings.append(rating)
        except:
            product_ratings.append(None)
 
        try:
            # Number of Reviews
            reviews = product.find_element(By.CLASS_NAME, "Wphh3N").text
            num_reviews.append(reviews)
        except:
            num_reviews.append(None)
 
        try:
            # Discount
            discount = product.find_element(By.CLASS_NAME, "UkUFwK").text
            discounts.append(discount)
        except:
            discounts.append(None)
scrape_page()
 
# Print some sample data to check
#print(f"Product Names: {product_names[:5]}")
#print(f"Product Prices: {product_prices[:5]}")

In [9]:
# Number of pages to scrape
pages_to_scrape = 25
current_page = 1
 
while current_page < pages_to_scrape:
    try:
        # Find and click the "Next" button to go to the next page
        next_button = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[12]/div/div/nav/a[11]')
        ActionChains(driver).move_to_element(next_button).click().perform()
        time.sleep(1)  # Wait for the next page to load
 
        # Scrape the next page's products
        scrape_page()
 
        # Increment page counter
        current_page += 1
 
    except Exception as e:
        print("Error or no more pages:", e)
        break  # Exit the loop if no "Next" button is found or an error occurs
 
# Print some data after scraping multiple pages
print(f"Scraped {len(product_names)} products.")

Scraped 1000 products.


In [3]:
data = {
    "Product Name": product_names,
    "Price": product_prices,
    "Rating": product_ratings,
    "Number of Reviews": num_reviews,
    "Discount": discounts
}
 
df = pd.DataFrame(data)
df

NameError: name 'product_names' is not defined

In [2]:
df.to_csv('flipkart_best_selling_toys.csv', index=False)

NameError: name 'df' is not defined

In [19]:
df1=df.dropna(axis=0)

In [24]:
df1['Discount'] = df1['Discount'].str.replace(' off', '', regex=False)
df1

,Product Name,Price,Rating,Number of Reviews,Discount
0,ZRIYOS 8.5 inch LCD Writing tab For Kids Re-Wr...,₹239,3.7,(588),65%
1,Toys R Us Avigo 888 KIDS Car Battery Operated ...,"₹7,749",3.9,(663),69%
2,KADAM Pakka Rang Permanent Fabric Dye Colour |...,₹186,4.1,"(7,494)",38%
3,KADAM Pakka Rang Permanent Fabric Dye Colour |...,₹186,4,"(5,767)",38%
4,"TAPUJI Dancing Cactus Repeats What You Say,Ele...",₹243,3.9,"(74,107)",87%
...,...,...,...,...,...
995,Parksons 2 in 1 - Snakes & Ladder and Ludo Dar...,₹161,3.8,(235),51%
996,Rilson Cast Iron Heavy Duty Fabric Button Maki...,"₹1,749",4.4,(602),45%
997,Planet of Toys Heavy Duty Bow Arrow Set for Ki...,₹469,4,(648),53%
998,JANCOM Sand Activity Play Set Natural Kinetic ...,₹549,4.3,(420),45%


In [28]:
df1['Number of Reviews'] = df1['Number of Reviews'].str.replace(r'[()]', '', regex=True)
df1

,Product Name,Price,Rating,Number of Reviews,Discount
0,ZRIYOS 8.5 inch LCD Writing tab For Kids Re-Wr...,₹239,3.7,588,65%
1,Toys R Us Avigo 888 KIDS Car Battery Operated ...,"₹7,749",3.9,663,69%
2,KADAM Pakka Rang Permanent Fabric Dye Colour |...,₹186,4.1,"7,494",38%
3,KADAM Pakka Rang Permanent Fabric Dye Colour |...,₹186,4,"5,767",38%
4,"TAPUJI Dancing Cactus Repeats What You Say,Ele...",₹243,3.9,"74,107",87%
...,...,...,...,...,...
995,Parksons 2 in 1 - Snakes & Ladder and Ludo Dar...,₹161,3.8,235,51%
996,Rilson Cast Iron Heavy Duty Fabric Button Maki...,"₹1,749",4.4,602,45%
997,Planet of Toys Heavy Duty Bow Arrow Set for Ki...,₹469,4,648,53%
998,JANCOM Sand Activity Play Set Natural Kinetic ...,₹549,4.3,420,45%


In [31]:
df1['Rating'] = pd.to_numeric(df1['Rating'], errors='coerce')
df1['Rating Category'] = df1['Rating'].apply(
    lambda x: 'Very Good' if x > 4 else ('Good' if x > 3 else 'Average')
)
df1

,Product Name,Price,Rating,Number of Reviews,Discount,Rating Category
0,ZRIYOS 8.5 inch LCD Writing tab For Kids Re-Wr...,₹239,3.7,588,65%,Good
1,Toys R Us Avigo 888 KIDS Car Battery Operated ...,"₹7,749",3.9,663,69%,Good
2,KADAM Pakka Rang Permanent Fabric Dye Colour |...,₹186,4.1,"7,494",38%,Very Good
3,KADAM Pakka Rang Permanent Fabric Dye Colour |...,₹186,4.0,"5,767",38%,Good
4,"TAPUJI Dancing Cactus Repeats What You Say,Ele...",₹243,3.9,"74,107",87%,Good
...,...,...,...,...,...,...
995,Parksons 2 in 1 - Snakes & Ladder and Ludo Dar...,₹161,3.8,235,51%,Good
996,Rilson Cast Iron Heavy Duty Fabric Button Maki...,"₹1,749",4.4,602,45%,Very Good
997,Planet of Toys Heavy Duty Bow Arrow Set for Ki...,₹469,4.0,648,53%,Good
998,JANCOM Sand Activity Play Set Natural Kinetic ...,₹549,4.3,420,45%,Very Good


In [33]:
current_utc_time = datetime.utcnow()
df1['Timestamp'] = current_utc_time
df1

,Product Name,Price,Rating,Number of Reviews,Discount,Rating Category,Timestamp
0,ZRIYOS 8.5 inch LCD Writing tab For Kids Re-Wr...,₹239,3.7,588,65%,Good,2024-11-13 13:41:40.560958
1,Toys R Us Avigo 888 KIDS Car Battery Operated ...,"₹7,749",3.9,663,69%,Good,2024-11-13 13:41:40.560958
2,KADAM Pakka Rang Permanent Fabric Dye Colour |...,₹186,4.1,"7,494",38%,Very Good,2024-11-13 13:41:40.560958
3,KADAM Pakka Rang Permanent Fabric Dye Colour |...,₹186,4.0,"5,767",38%,Good,2024-11-13 13:41:40.560958
4,"TAPUJI Dancing Cactus Repeats What You Say,Ele...",₹243,3.9,"74,107",87%,Good,2024-11-13 13:41:40.560958
...,...,...,...,...,...,...,...
995,Parksons 2 in 1 - Snakes & Ladder and Ludo Dar...,₹161,3.8,235,51%,Good,2024-11-13 13:41:40.560958
996,Rilson Cast Iron Heavy Duty Fabric Button Maki...,"₹1,749",4.4,602,45%,Very Good,2024-11-13 13:41:40.560958
997,Planet of Toys Heavy Duty Bow Arrow Set for Ki...,₹469,4.0,648,53%,Good,2024-11-13 13:41:40.560958
998,JANCOM Sand Activity Play Set Natural Kinetic ...,₹549,4.3,420,45%,Very Good,2024-11-13 13:41:40.560958


In [1]:
df1.to_csv('flipkart_best_selling1.csv', index=False)

NameError: name 'df1' is not defined

In [43]:
!pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable


In [45]:
!pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
  Using cached psycopg2-2.9.10.tar.gz (385 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp310-cp310-linux_x86_64.whl size=499326 sha256=bc38b578cf1a74d060327d3f8d042e244dab5d052591b528e1398647b20a3e4a
  Stored in directory: /home/mt24048/.cache/pip/wheels/51/41/e0/2912ad51b01f454d26dfb26e5cc5923874656749b9e83943a8
Successfully built psycopg2


In [1]:
!pip install sqlalchemy psycopg2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

In [6]:
df1 = pd.read_csv('/home/mt24048/Documents/flipkart_best_selling.csv')

In [8]:
df1['Price'] = df1['Price'].replace({'₹': '', ',': ''}, regex=True).astype(float)
df1

,Product Name,Price,Rating,Number of Reviews,Discount,Rating Category,Timestamp
0,ZRIYOS 8.5 inch LCD Writing tab For Kids Re-Wr...,239.0,3.7,588,65%,Good,2024-11-13 13:41:40.560958
1,Toys R Us Avigo 888 KIDS Car Battery Operated ...,7749.0,3.9,663,69%,Good,2024-11-13 13:41:40.560958
2,KADAM Pakka Rang Permanent Fabric Dye Colour |...,186.0,4.1,"7,494",38%,Very Good,2024-11-13 13:41:40.560958
3,KADAM Pakka Rang Permanent Fabric Dye Colour |...,186.0,4.0,"5,767",38%,Good,2024-11-13 13:41:40.560958
4,"TAPUJI Dancing Cactus Repeats What You Say,Ele...",243.0,3.9,"74,107",87%,Good,2024-11-13 13:41:40.560958
...,...,...,...,...,...,...,...
953,Parksons 2 in 1 - Snakes & Ladder and Ludo Dar...,161.0,3.8,235,51%,Good,2024-11-13 13:41:40.560958
954,Rilson Cast Iron Heavy Duty Fabric Button Maki...,1749.0,4.4,602,45%,Very Good,2024-11-13 13:41:40.560958
955,Planet of Toys Heavy Duty Bow Arrow Set for Ki...,469.0,4.0,648,53%,Good,2024-11-13 13:41:40.560958
956,JANCOM Sand Activity Play Set Natural Kinetic ...,549.0,4.3,420,45%,Very Good,2024-11-13 13:41:40.560958


In [10]:
df1['Rating'] = df1['Rating'].astype(float)
df1

,Product Name,Price,Rating,Number of Reviews,Discount,Rating Category,Timestamp
0,ZRIYOS 8.5 inch LCD Writing tab For Kids Re-Wr...,239.0,3.7,588,65%,Good,2024-11-13 13:41:40.560958
1,Toys R Us Avigo 888 KIDS Car Battery Operated ...,7749.0,3.9,663,69%,Good,2024-11-13 13:41:40.560958
2,KADAM Pakka Rang Permanent Fabric Dye Colour |...,186.0,4.1,"7,494",38%,Very Good,2024-11-13 13:41:40.560958
3,KADAM Pakka Rang Permanent Fabric Dye Colour |...,186.0,4.0,"5,767",38%,Good,2024-11-13 13:41:40.560958
4,"TAPUJI Dancing Cactus Repeats What You Say,Ele...",243.0,3.9,"74,107",87%,Good,2024-11-13 13:41:40.560958
...,...,...,...,...,...,...,...
953,Parksons 2 in 1 - Snakes & Ladder and Ludo Dar...,161.0,3.8,235,51%,Good,2024-11-13 13:41:40.560958
954,Rilson Cast Iron Heavy Duty Fabric Button Maki...,1749.0,4.4,602,45%,Very Good,2024-11-13 13:41:40.560958
955,Planet of Toys Heavy Duty Bow Arrow Set for Ki...,469.0,4.0,648,53%,Good,2024-11-13 13:41:40.560958
956,JANCOM Sand Activity Play Set Natural Kinetic ...,549.0,4.3,420,45%,Very Good,2024-11-13 13:41:40.560958


In [15]:
df1['Number of Reviews'] = df1['Number of Reviews'].astype(int)

In [16]:
df1

,Product Name,Price,Rating,Number of Reviews,Discount,Rating Category,Timestamp
0,ZRIYOS 8.5 inch LCD Writing tab For Kids Re-Wr...,239.0,3.7,588,65%,Good,2024-11-13 13:41:40.560958
1,Toys R Us Avigo 888 KIDS Car Battery Operated ...,7749.0,3.9,663,69%,Good,2024-11-13 13:41:40.560958
2,KADAM Pakka Rang Permanent Fabric Dye Colour |...,186.0,4.1,7494,38%,Very Good,2024-11-13 13:41:40.560958
3,KADAM Pakka Rang Permanent Fabric Dye Colour |...,186.0,4.0,5767,38%,Good,2024-11-13 13:41:40.560958
4,"TAPUJI Dancing Cactus Repeats What You Say,Ele...",243.0,3.9,74107,87%,Good,2024-11-13 13:41:40.560958
...,...,...,...,...,...,...,...
953,Parksons 2 in 1 - Snakes & Ladder and Ludo Dar...,161.0,3.8,235,51%,Good,2024-11-13 13:41:40.560958
954,Rilson Cast Iron Heavy Duty Fabric Button Maki...,1749.0,4.4,602,45%,Very Good,2024-11-13 13:41:40.560958
955,Planet of Toys Heavy Duty Bow Arrow Set for Ki...,469.0,4.0,648,53%,Good,2024-11-13 13:41:40.560958
956,JANCOM Sand Activity Play Set Natural Kinetic ...,549.0,4.3,420,45%,Very Good,2024-11-13 13:41:40.560958


In [19]:
# Connect to PostgreSQL using psycopg2
try:
    conn = psycopg2.connect(
        host="w3.training5.modak.com",
        database="postgres",
        user="mt24048",
        password="mt24048@m10y24",
        port="5432"
    )
    print("Connection successful.")
except Exception as e:
    print(f"An error occurred while connecting to the database: {e}")
 
# Create SQLAlchemy engine
engine = create_engine("postgresql+psycopg2://", creator=lambda: conn)
 
# Load your DataFrame into PostgreSQL
try:
    # Assuming `data` is your DataFrame
    df1.to_sql('mt24048_Flipkart_data', con=engine, index=False, if_exists='replace')
    print("Data loaded successfully into PostgreSQL.")
except Exception as e:
    print(f"An error occurred while loading data: {e}")
 
# Close the connection
conn.close()

Connection successful.
Data loaded successfully into PostgreSQL.


In [38]:
df1.to_csv('flipkart_best_selling.csv', index=False)